In [49]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import json
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import brain_drive_resps_dir, repo_dir
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# https://github.com/neurosynth/neurosynth-data/blob/master/data-neurosynth_version-7_vocab-terms_vocabulary.txt

In [ ]:
df = pd.read_csv('data-neurosynth_version-7_vocab-terms_vocabulary.txt')

In [40]:
vals = df['001'].str.strip()
assert len(vals) == len(vals.unique())
print(len(vals))
# vals remove any terms that contain numbers
vals = vals[~vals.str.contains(r'\d')]
print(len(vals))
vals = vals[vals.str.len() > 2]
print(len(vals))

# remove val if val + 's' in vals
vals = vals[~vals.apply(lambda x: (x + 's') in vals.values)]
print(len(vals))
vals = vals[~vals.apply(lambda x: (x + 'ly') in vals.values)]
print(len(vals))

3227
3160
3132
2816
2744


In [42]:
vals.to_csv('terms_full.csv')

In [47]:
prompt = '''You are given a comma-separated list of words / phrases. Your task is to classify whether each element of the list is a semantic text property or not. Some of the words instead describe properties of brain regions or diseases. Answer yes or no and return your answer in json.


### Example input
Input: abstract, acc, add, addiction, alcohol, anger, arithmetic, baseline, experiments, family, foot

### Example output
```
{
  "abstract": "yes",
  "acc": "no",
  "addiction": "yes",
  "alcohol": "no",
  "anger": "yes",
  "arithmetic": "yes",
  "baseline": "no",
  "date": "yes",
  "debate": "yes",
  "dti": "no",
  "exhibit": "no",
  "experiments": "no",
  "family": "yes",
  "foot": "yes"
}
```

### Input
'''

In [46]:
', '.join(vals.values)

'aberrant, abilities, ability, able, abnormal, abnormalities, abnormality, absence, absent, abstract, abuse, acc, access, accompanied, accordance, accordingly, accounted, accounts, accumbens, accuracy, accurately, achieve, achieved, acoustic, acquired, acquisition, action observation, actions, activates, activations, actively, activities, acts, actually, acute, adaptation, adapted, adaptive, add, addiction, addition, additionally, addressed, adhd, adjacent, administered, administration, adolescence, adolescents, adopted, adulthood, adults, advanced, advances, advantage, affected, affecting, affective, affects, age controls, age gender, age groups, age healthy, age sex, aged, agent, ages, aging, agreement, aimed, aims, alcohol, alff, algorithm, allele, allocation, allowed, allowing, allows, alpha, alterations, altered, alternating, alternative, alters, alzheimer, alzheimer disease, ambiguous, amnestic, amplitude low, amplitudes, amygdala, amygdala anterior, amygdala hippocampus, amygdal

In [65]:
filt1 = json.load(open('terms_filt1.json', 'r'))

In [66]:
vals_filt1 = sorted({k for k in filt1 if filt1[k] == 'yes'})

In [67]:
pd.DataFrame(vals_filt1).to_csv('terms_filt1.csv')

In [68]:
len(vals), len(vals_filt1)

(2744, 534)

In [70]:
', '.join(vals_filt1)

'abilities, ability, able, abstract, access, accordance, accordingly, accuracy, accurately, achieve, achieved, action observation, actions, actively, activities, acts, actually, adaptation, addition, additionally, advantage, agreement, allocation, alternative, ambiguous, anger, angry, anticipation, apparent, application, apply, appraisal, approach, appropriate, approximately, architecture, argue, arise, arithmetic, aspects, assess, assume, assumption, attempt, attend, attention, attentional control, attribution, automatic, automatically, available, average, avoid, avoidance, awareness, background, balance, basic, basis, beginning, behavioral evidence, behavioral level, behavioral measures, behavioral performance, behavioral responses, behaviorally, behaviors, behaviour, behavioural, beliefs, beneficial, benefits, best, better, better understanding, bias, biased, biases, brief, broad, broader, calculation, capacity, capture, categorical, categories, categorization, category, causal, cau